In [9]:
import tensorflow_datasets as tfds
import tensorflow as tf

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name = "imdb_reviews",
    split = ["train[:90%]", "train[90%:]", "test"],
    as_supervised= True
)

tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [10]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8"))
    print("Label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development

In [11]:
train_set.map(lambda reviews, labels:reviews)

<MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [12]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels:reviews))


In [13]:
# embed_size = 128

# tf.random.set_seed(42)
# model = tf.keras.Sequential([
#     text_vec_layer,
#     tf.keras.layers.Embedding(vocab_size, embed_size),
#     tf.keras.layers.GRU(128),
#     tf.keras.layers.Dense(1, activation="sigmoid")
# ])

# model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

# history = model.fit(train_set, validation_data=valid_set, epochs=2)

In [14]:
embed_size = 128

inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
token_ids = text_vec_layer(inputs)
mask = tf.math.not_equal(token_ids, 0)
Z = tf.keras.layers.Embedding(vocab_size, embed_size)(token_ids)
Z = tf.keras.layers.GRU(128, dropout=0.2)(Z, mask=mask)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(Z)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 194s 273ms/step - loss: 0.5018 - accuracy: 0.7514 - val_loss: 0.8501 - val_accuracy: 0.6732
Epoch 2/2
704/704 [==============================] - 185s 263ms/step - loss: 0.3261 - accuracy: 0.8637 - val_loss: 0.3952 - val_accuracy: 0.8280


In [15]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(max_tokens=vocab_size, ragged=True)

text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels:reviews))
text_vec_layer_ragged(["Great movie!", "This is DiCaprio's best role."])


<tf.RaggedTensor [[86, 18], [11, 7, 1, 116, 217]]>

In [16]:
text_vec_layer(["Great movie!", "This is DiCaprio's best role."])

<tf.Tensor: shape=(2, 5), dtype=int64, numpy=
array([[ 86,  18,   0,   0,   0],
       [ 11,   7,   1, 116, 217]], dtype=int64)>

In [17]:
import os
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"

model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string,input_shape=[]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit(train_set,validation_data=valid_set, epochs=10)

Epoch 1/10
704/704 [==============================] - 745s 1s/step - loss: 0.3124 - accuracy: 0.8669 - val_loss: 0.2766 - val_accuracy: 0.8856
Epoch 2/10
704/704 [==============================] - 738s 1s/step - loss: 0.0493 - accuracy: 0.9852 - val_loss: 0.3013 - val_accuracy: 0.8944
Epoch 3/10
704/704 [==============================] - 735s 1s/step - loss: 0.0049 - accuracy: 0.9991 - val_loss: 0.3818 - val_accuracy: 0.8908
Epoch 4/10
 94/704 [===>..........................] - ETA: 10:41 - loss: 0.0010 - accuracy: 0.9997

KeyboardInterrupt: 